In [1]:
import joblib
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn

import yaml

from load_data import split_data

class BestModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.gru_X = nn.GRU(
            input_size=7 * 24,
            hidden_size=hidden_size,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(hidden_size, 24)

    def forward(self, x):
        x, _ = self.gru_X(x)
        x = self.linear(x)
        return x

X_scaler = joblib.load("X_scaler.joblib")
model = torch.load("torch_LSTM.pickle")

with open("params.yaml", "r") as yaml_file:
    file_paths = yaml.safe_load(yaml_file)
        
data = pd.read_csv(
    file_paths["active_loss_path"],
    skiprows=1,
    names=["unnamed0", "unnamed1", "timestamps", "MWh"],
    parse_dates=["timestamps"],
    index_col="timestamps",
)
data = data.drop(columns=["unnamed0", "unnamed1"])
data["MWh"] = data["MWh"]


data

/home/jupyter-user05/.local/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,MWh
timestamps,
2022-01-01 00:00:00,146.054792
2022-01-01 01:00:00,139.133354
2022-01-01 02:00:00,147.562500
2022-01-01 03:00:00,157.636204
2022-01-01 04:00:00,163.326766
...,...
2022-12-31 19:00:00,67.876028
2022-12-31 20:00:00,72.765318
2022-12-31 21:00:00,81.277633


In [2]:
X, y = split_data(data, 7 * 24, 24, 1, "MWh")

X.shape, y.shape

((8569, 168), (8569, 24))

In [3]:
X = X_scaler.transform(X)

In [4]:
X = torch.Tensor(X)[:, :].to(device='cuda')
y = torch.Tensor(y)[:, :].to(device='cuda')

In [5]:
## import torch.optim as optim
from torch import utils

loader = utils.data.DataLoader(
    utils.data.TensorDataset(X, y),
    shuffle=False,
    batch_size=1024,
)
    
model.eval()
with torch.no_grad():
    y_pred = model(X)
    mae = nn.L1Loss()(y_pred, y).item()
print("MAE %.4f" % (mae))

MAE 22.0318


In [6]:
y_pred.shape

torch.Size([8569, 24])

In [7]:
y_pred_np = y_pred.to(device="cpu").numpy() #convert to Numpy array
y_pred_df = pd.DataFrame(y_pred_np) #convert to a dataframe
y_pred_df.to_csv("y_pred.csv",index=False) #save to file